### Set Environment

In [67]:
import json

### List all Output paths

In [68]:
from glob2 import glob
jsonFiles = glob('/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/*/posts/*.json') #Can be used absolute or relative paths

### Merge all data:

In [64]:
def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
        with open(f1, 'r') as infile:
            result.extend(json.load(infile))

    with open('counseling3.json', 'w') as output_file:
        json.dump(result, output_file)

merge_JsonFiles(jsonFiles)

### Save as .csv

In [66]:
result.to_csv("/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/tiktok.csv", index=False)

,id,secretID,text,createTime,authorMeta,musicMeta,covers,webVideoUrl,videoUrl,videoUrlNoWaterMark,videoApiUrlNoWaterMark,videoMeta,diggCount,shareCount,playCount,commentCount,downloaded,mentions,hashtags,effectStickers
0,7053605558787673088,7053605558787673088,@ohiostatedanceteam and @shannonmather1 I am y...,1642295523,"{'id': '6637452456970158086', 'secUid': 'MS4wL...","{'musicId': '7029929128363035398', 'musicName'...",{'default': 'https://p19-sign.tiktokcdn-us.com...,https://www.tiktok.com/@hagsjkdndusgaveneoh/vi...,https://v16-webapp.tiktok.com/6cc186782203f560...,,,"{'height': 1056, 'width': 576, 'duration': 10}",607700,4266,4100000,3653,False,"[@ohiostatedanceteam, @shannonmather1]","[{'id': '378575', 'name': 'uda', 'title': '', ...",[]
1,7053531644015201280,7053531644015201280,The mental part of this isnt it on the last da...,1642278313,"{'id': '6681327745416840197', 'secUid': 'MS4wL...","{'musicId': '7053531629486295854', 'musicName'...",{'default': 'https://p16-sign.tiktokcdn-us.com...,https://www.tiktok.com/@ryleagoodson/video/705...,https://v16-webapp.tiktok.com/d5c056bef893e2a2...,,,"{'height': 1024, 'width': 576, 'duration': 85}",543900,1653,4600000,2016,False,[],"[{'id': '378575', 'name': 'uda', 'title': '', ...",[]
2,6829568070978112512,6829568070978112512,#udad #bete #adarolage#uda #ede#🐊🐊🐊,1590132733,"{'id': '6607623898308313093', 'secUid': 'MS4wL...","{'musicId': '6625840862708370181', 'musicName'...",{'default': 'https://p16-sign-va.tiktokcdn.com...,https://www.tiktok.com/@ajay_bhajantri/video/6...,https://v16-webapp.tiktok.com/dfc4ca4e47952bf5...,,,"{'height': 640, 'width': 352, 'duration': 15}",152200,305,2200000,76,False,[],"[{'id': '51634732', 'name': 'udad', 'title': '...",[]
3,7054957844436683776,7054957844436683776,*my whole house every 5 minutes* DC: @tribe.99...,1642610377,"{'id': '7925210', 'secUid': 'MS4wLjABAAAAcj_1N...","{'musicId': '242459444368871424', 'musicName':...",{'default': 'https://p16-sign.tiktokcdn-us.com...,https://www.tiktok.com/@marleyblanchard/video/...,https://v16-webapp.tiktok.com/899d28e182264e45...,,,"{'height': 1024, 'width': 576, 'duration': 13}",116800,568,1100000,331,False,"[@tribe, @LSU, @ninagandrianos]","[{'id': '51009', 'name': 'osu', 'title': '', '...","[{'id': '263840', 'name': 'Green Screen'}]"
4,7069199295848713216,7069199295848713216,#34 hari lagi#bulanramadhan #2022#uda gk sabar...,1645926223,"{'id': '6968190259868402689', 'secUid': 'MS4wL...","{'musicId': '7069199303331400474', 'musicName'...",{'default': 'https://p16-sign-va.tiktokcdn.com...,https://www.tiktok.com/@kangstoriwa8/video/706...,https://v16-webapp.tiktok.com/ff7b114e4363e3f3...,,,"{'height': 1024, 'width': 576, 'duration': 24}",99200,6334,785800,3178,False,[],"[{'id': '38084783', 'name': '34', 'title': '',...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,7056919275570875392,7056919275570875392,¿quiénes no son verdaderos gemelos? #sliimecak...,1643067058,"{'id': '7039539693809730565', 'secUid': 'MS4wL...","{'musicId': '7056919321372625670', 'musicName'...",{'default': 'https://p16-sign-va.tiktokcdn.com...,https://www.tiktok.com/@sliime..cakee/video/70...,https://v16-webapp.tiktok.com/1fa9fc0f118e92a6...,,,"{'height': 1024, 'width': 576, 'duration': 180}",395200,201,2200000,668,False,[],"[{'id': '7039363925422374918', 'name': 'sliime...",[]
26,6914732689333488640,6914732689333488640,SPECTRUM: Christian ⛪️#Christian #lgbtq #empat...,1609961678,"{'id': '6736753654080537605', 'secUid': 'MS4wL...","{'musicId': '6914732686959512325', 'musicName'...",{'default': 'https://p16-sign-va.tiktokcdn.com...,https://www.tiktok.com/@jubilee/video/69147326...,https://v16-webapp.tiktok.com/38b78addf17ddd4d...,,,"{'height': 1024, 'width': 576, 'duration': 44}",362900,4492,1100000,6956,False,[],"[{'id': '46249', 'name': 'christian', 'title':...",[]
27,7104933450691480576,7104933450691480576,‘And can you dance like you’re being held host...,1654246238,"{'id': '665371163132197